[dataset link](https://github.com/udacity/deep-learning/tree/master/intro-to-rnns)

# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
print(len(vocab))
vocab

83


{'\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [4]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [21]:
type(vocab)# all unique characters including symbols and punct # how to access individual element of a set?
type(vocab_to_int) # all unique characters paired with index # how to access index and values of a dictionary?
int_to_vocab # index paired with characters
len(chars) # total number of characters of the whole text

1985223

In [22]:
chars[:100]

array([16, 50, 73,  0, 42,  4,  3, 40,  6, 71, 71, 71, 26, 73,  0,  0, 23,
       40,  8, 73, 39, 55, 46, 55,  4, 74, 40, 73,  3,  4, 40, 73, 46, 46,
       40, 73, 46, 55,  2,  4,  1, 40,  4, 41,  4,  3, 23, 40, 47,  7, 50,
       73,  0,  0, 23, 40,  8, 73, 39, 55, 46, 23, 40, 55, 74, 40, 47,  7,
       50, 73,  0,  0, 23, 40, 55,  7, 40, 55, 42, 74, 40,  9, 81,  7, 71,
       81, 73, 23, 10, 71, 71, 54, 41,  4,  3, 23, 42, 50, 55,  7], dtype=int32)

## Making training and validation batches


In [17]:
np.split(np.array([2,3,4,5,6,7]), 3)#.shape
# np.stack(np.split(np.array([2,3,4,5,6,7]), 3))
# np.stack(np.split(np.array([2,3,4,5,6,7]), 3)).shape

[array([2, 3]), array([4, 5]), array([6, 7])]

In [19]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
                                            # 1 character * num_steps(meaning: how many chars as 1 sample) = 1 sample 
                                            # 1 sample * batch_size(meaning: how many samples as 1 batch)= 1 batch = 1 slice_size 
    slice_size = batch_size * num_steps
    
                                            # 1 batch or 1 slice_size * num_batches = total chars = a whole text
                                            # int(6.02) == 6, drop a little rest
    n_batches = int(len(chars) / slice_size)
    
                                            # Drop the last few characters to make only full batches
                                            # y is shift 1 char right to x
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    
                                            # np.split(np.array([2,3,4,5,6,7]), 3) = split into a list of 3 arrays, 
                                            # each array has 2 elements
                                            # np.stack(): turn list into 2-d array (3,2)
                                            # now: x is 2-d array dim = (batch_size, x/batch_size) 
                                            # now: y is 2-d array dim = (batch_size, y/batch_size) 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    
    
                                            # split into train and valid sets: split_frac is train, rest is valid
                                            # split not by rows, but split on columns, 10 rows remains 10 rows = batch_size
                                            # total chars = 1 char * num_steps * batch_size * num_batches, since batch_size stays
                                            # num_train_columns = split_frac * (num_batches * num_steps)
                                            # int() to remove the little rest
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [23]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 50)

In [27]:
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

(10, 178650)
(10, 178650)
(10, 19850)
(10, 19850)


**Check 1st batch**: 
- 1 batch = batch_size * 1 sample = batch_size * (1 char * num_step)
- batch_size = 10
- num_step = 50

In [28]:
train_x[:,:50]

array([[16, 50, 73,  0, 42,  4,  3, 40,  6, 71, 71, 71, 26, 73,  0,  0, 23,
        40,  8, 73, 39, 55, 46, 55,  4, 74, 40, 73,  3,  4, 40, 73, 46, 46,
        40, 73, 46, 55,  2,  4,  1, 40,  4, 41,  4,  3, 23, 40, 47,  7],
       [40, 73, 39, 40,  7,  9, 42, 40, 11,  9, 55,  7, 11, 40, 42,  9, 40,
        74, 42, 73, 23, 52, 65, 40, 73,  7, 74, 81,  4,  3,  4, 13, 40, 43,
         7,  7, 73, 52, 40, 74, 39, 55, 46, 55,  7, 11, 52, 40, 75, 47],
       [41, 55,  7, 10, 71, 71, 65, 68,  4, 74, 52, 40, 55, 42, 48, 74, 40,
        74,  4, 42, 42, 46,  4, 13, 10, 40, 17, 50,  4, 40,  0,  3, 55, 12,
         4, 40, 55, 74, 40, 39, 73, 11,  7, 55,  8, 55, 12,  4,  7, 42],
       [ 7, 40, 13, 47,  3, 55,  7, 11, 40, 50, 55, 74, 40, 12,  9,  7, 41,
         4,  3, 74, 73, 42, 55,  9,  7, 40, 81, 55, 42, 50, 40, 50, 55, 74,
        71, 75,  3,  9, 42, 50,  4,  3, 40, 81, 73, 74, 40, 42, 50, 55],
       [40, 55, 42, 40, 55, 74, 52, 40, 74, 55,  3, 53, 65, 40, 74, 73, 55,
        13, 40, 42, 50, 


**split train_x, train_y into batches**: 
- arrs: [train_x, train_y] or [val_x, val_y]
- slice_size is a misleading name: its actual identity == x/batch_size == num_steps * n_batches


In [32]:
                                        # - arrs == [train_x, train_y] or [val_x, val_y]
def get_batch(arrs, num_steps):
    
                                        # - slice_size is a misleading name: its actual identity == x/batch_size
                                                                                               # == num_steps * n_batches
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    
                                        # loop each batch
    for b in range(n_batches):
                                        # loop between [train_x, train_y]
                                        # train_x, for 1st batch, keep all rows (10 rows total), select columns from 0 to 49
                                        # train_y, for 1st batch, keep all rows (10 rows total), select columns from 0 to 49
                                        # train_x, for 2nd batch, keep all rows (10 rows total), select columns from 50 to 99
                                        # train_y, for 2nd batch, keep all rows (10 rows total), select columns from 50 to 99
                                        # yield: is to put all above into an object iterable
                    # https://hyp.is/dJ-VvgSWEeeGA_P8j321Fg/pythontips.com/2013/09/29/the-python-yield-keyword-explained/
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]
        
                                        # yield help split train_x into n_batches of batches, each with dim (10, 50)

## Building the model

Below is a function where I build the graph for the network.

In [24]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
    
                                                # set sampling = True, this way, we only feed 1 char to model at a time
                                                # this can speed up 
    if sampling == True:
        batch_size, num_steps = 1, 1

        
        
    tf.reset_default_graph()
    
    
                                                # Declare placeholders for inputs and targets
                                                # inputs and targets are tensors, each with dim (batch_size, num_steps)
                                                # there are n_batches of these tensors in total
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    
                                                # define probability placeholder to feed in dropout value
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    
                                                # One-hot encoding the input and target tensors
                                                # now tensor is 3-d (batch_size, num_steps, vocab)
    x_one_hot = tf.one_hot(inputs, num_classes)
    y_one_hot = tf.one_hot(targets, num_classes)

    
    
                                                ### Build the RNN layers
                                                # Use a basic LSTM cell with 128 neurons
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    
                                                # Add dropout to the cell, with dropout value placehoder added
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    
                                                # Stack up multiple LSTM layers, here are 2 layers required
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    
                                                # initialize the state with zero state
    initial_state = cell.zero_state(batch_size, tf.float32)


    
    
    
                                                ### turn x_one_hot 3-d into rnn_inputs as 1-d???????
                                                
                                                # tf.split: split x_one_hot (batch_size, num_steps, vocab) into 
                                                # num_steps of 2-d tenors, each with dim (batch_size, vocab)
                                                # tf.squeeze(tensor, axis=[1]), to squeeze off second dim
                                                # now we have a list of num_steps of 1-d tensor with dim (batch_size*vocab, )
                            
                            # https://hyp.is/Gt7csASbEeeNUk9ozR_ZuA/www.tensorflow.org/api_docs/python/tf/split
                            # https://hyp.is/Pmiw-ASaEeeMJkOMBxWpOw/www.tensorflow.org/api_docs/python/tf/squeeze
    rnn_inputs = [tf.squeeze(i, axis=[1]) for i in tf.split(x_one_hot, num_steps, 1)]
                                                # now rnn_inputs is a list of num_steps of column vectors
    
    
                                                # Run each sequence step (num_steps in total) through the RNN and 
                                                # collect the outputs
    outputs, state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state=initial_state)
    final_state = state
    
    
    # Reshape output so it's a bunch of rows, one output row for each step for each batch
    
                                                # outputs are outputs of all hidden layers,
                                                # tf.concat to join them together into one large output
                                                # tf.reshape into a large output tensor dim (-1, lstm_size)
    seq_output = tf.concat(outputs, axis=1)
    output = tf.reshape(seq_output, [-1, lstm_size])
    
    
    
                                                ### build a softmax layer
                                                # build softmax_w and softmax_b variable holders
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(num_classes))
        
    
                                                # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
                                                # of rows of logit outputs, one for each step and batch
    logits = tf.matmul(output, softmax_w) + softmax_b
    
                                                # Use softmax to get the probabilities for predicted characters
    preds = tf.nn.softmax(logits, name='predictions')
    
    
                                                # Reshape the targets to match the logits for calc cross-entropy
    y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    
                                                # calc loss with cross-entropy
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    cost = tf.reduce_mean(loss)

    
    
                                                # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    
    
    # Export the nodes using namedtuple
    # https://hyp.is/3dAyfgSfEee9XvuDFXgZZA/pymotw.com/2/collections/namedtuple.html
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    
                                                # use to use locals()
# https://hyp.is/iQd3kgSgEeei6T9SRPMc5A/stackoverflow.com/questions/7969949/whats-the-difference-between-globals-locals-and-vars
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to write it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that heps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


In [26]:
batch_size = 100
num_steps = 100 
lstm_size = 512
num_layers = 2
learning_rate = 0.001
keep_prob = 0.5

## Training

Time for training which is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}_v{validation loss}.ckpt`

In [29]:
epochs = 20
# Save every N iterations
save_every_n = 200
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)



with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer], 
                                                 feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
        
            
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/i{}_l{}_v{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

Epoch 1/20  Iteration 1/3560 Training loss: 4.4176 9.2723 sec/batch
Epoch 1/20  Iteration 2/3560 Training loss: 4.3724 6.3679 sec/batch
Epoch 1/20  Iteration 3/3560 Training loss: 4.1883 6.5060 sec/batch
Epoch 1/20  Iteration 4/3560 Training loss: 4.4842 6.6540 sec/batch
Epoch 1/20  Iteration 5/3560 Training loss: 4.4509 6.3942 sec/batch
Epoch 1/20  Iteration 6/3560 Training loss: 4.3708 6.3062 sec/batch
Epoch 1/20  Iteration 7/3560 Training loss: 4.2867 6.2887 sec/batch
Epoch 1/20  Iteration 8/3560 Training loss: 4.2082 6.3095 sec/batch
Epoch 1/20  Iteration 9/3560 Training loss: 4.1324 6.2057 sec/batch
Epoch 1/20  Iteration 10/3560 Training loss: 4.0661 6.7661 sec/batch
Epoch 1/20  Iteration 11/3560 Training loss: 4.0063 7.0095 sec/batch
Epoch 1/20  Iteration 12/3560 Training loss: 3.9577 6.6022 sec/batch
Epoch 1/20  Iteration 13/3560 Training loss: 3.9159 6.7216 sec/batch
Epoch 1/20  Iteration 14/3560 Training loss: 3.8795 6.4497 sec/batch
Epoch 1/20  Iteration 15/3560 Training loss

KeyboardInterrupt: 

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [ ]:
tf.train.get_checkpoint_state('checkpoints')

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [17]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [41]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [ ]:
checkpoint = "checkpoints/____.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)